## Richter's Predictor: Modeling Earthquake Damage

The April 2015 earthquake with a magnitude of 7.8Mw in Nepal(aka Gorkha earthquake) had nearly 9,000 causalties and injured almost 22,000 people. Experts had long warned of the possible damages from deadly earthquakes in Nepal due to its geology, architecture and other factors. Many centuries-old buildings that were a UNESCO World Heritage Site were destroyed.

In this challenge, we will be predicting the level of damage to buildings ('damage grade'), on a scale of 1-3, with 1 being the lowest grade. This is a multiclass classification problem where we will solve by using logistic regression.

This is part of a Driven Data competition: https://www.drivendata.org/competitions/57/nepal-earthquake/

Feature descriptions can be found here: https://www.drivendata.org/competitions/57/nepal-earthquake/page/136/

In [1]:
#import useful libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt # for plotting
import seaborn as sns # for plotting

%matplotlib inline

import warnings # hide warning messages
warnings.filterwarnings('ignore')

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/matplotlib/font_manager.py:229: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


### Load Datasets

In [2]:
train = pd.read_csv('train_values.csv')
label = pd.read_csv('train_labels.csv')
test = pd.read_csv('test_values.csv')

In [3]:
#displays all columns
pd.set_option('display.max_columns', 80)
train.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,802906,6,487,12198,2,30,6,5,t,r,n,f,q,t,d,1,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
1,28830,8,900,2812,2,10,8,7,o,r,n,x,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
2,94947,21,363,8973,2,10,5,5,t,r,n,f,x,t,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
3,590882,22,418,10694,2,10,6,5,t,r,n,f,x,s,d,0,1,0,0,0,0,1,1,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
4,201944,11,131,1488,3,30,8,9,t,r,n,f,x,s,d,1,0,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0


In [4]:
test.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,300051,17,596,11307,3,20,7,6,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
1,99355,6,141,11987,2,25,13,5,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,1,1,0,0,0,0,0,0,0,0,0
2,890251,22,19,10044,2,5,4,5,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
3,745817,26,39,633,1,0,19,3,t,r,x,v,j,t,d,0,0,0,0,0,1,0,0,0,0,0,v,2,1,0,0,1,0,0,0,0,0,0,0
4,421793,17,289,7970,3,15,8,7,t,r,q,f,q,t,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0


##### Quick glances with statistical summary

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 39 columns):
building_id                               260601 non-null int64
geo_level_1_id                            260601 non-null int64
geo_level_2_id                            260601 non-null int64
geo_level_3_id                            260601 non-null int64
count_floors_pre_eq                       260601 non-null int64
age                                       260601 non-null int64
area_percentage                           260601 non-null int64
height_percentage                         260601 non-null int64
land_surface_condition                    260601 non-null object
foundation_type                           260601 non-null object
roof_type                                 260601 non-null object
ground_floor_type                         260601 non-null object
other_floor_type                          260601 non-null object
position                                  260601 non

In [6]:
train.describe()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
count,2.606010e+05,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000
mean,5.256755e+05,13.900353,701.074685,6257.876148,2.129723,26.535029,8.018051,5.434365,0.088645,0.761935,0.034332,0.018235,0.068154,0.075268,0.254988,0.085011,0.042590,0.015859,0.014985,0.983949,0.111880,0.064378,0.033626,0.008101,0.000940,0.000361,0.001071,0.000188,0.000146,0.000088,0.005119
std,3.045450e+05,8.033617,412.710734,3646.369645,0.727665,73.565937,4.392231,1.918418,0.284231,0.425900,0.182081,0.133800,0.252010,0.263824,0.435855,0.278899,0.201931,0.124932,0.121491,0.418389,0.315219,0.245426,0.180265,0.089638,0.030647,0.018989,0.032703,0.013711,0.012075,0.009394,0.071364
min,4.000000e+00,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.611900e+05,7.000000,350.000000,3073.000000,2.000000,10.000000,5.000000,4.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.257570e+05,12.000000,702.000000,6270.000000,2.000000,15.000000,7.000000,5.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.897620e+05,21.000000,1050.000000,9412.000000,2.000000,30.000000,9.000000,6.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.052934e+06,30.000000,1427.000000,12567.000000,9.000000,995.000000,100.000000,32.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,9.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86868 entries, 0 to 86867
Data columns (total 39 columns):
building_id                               86868 non-null int64
geo_level_1_id                            86868 non-null int64
geo_level_2_id                            86868 non-null int64
geo_level_3_id                            86868 non-null int64
count_floors_pre_eq                       86868 non-null int64
age                                       86868 non-null int64
area_percentage                           86868 non-null int64
height_percentage                         86868 non-null int64
land_surface_condition                    86868 non-null object
foundation_type                           86868 non-null object
roof_type                                 86868 non-null object
ground_floor_type                         86868 non-null object
other_floor_type                          86868 non-null object
position                                  86868 non-null object
pla

In [8]:
test.describe()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
count,8.686800e+04,86868.000000,86868.000000,86868.000000,86868.000000,86868.000000,86868.000000,86868.000000,86868.000000,86868.000000,86868.000000,86868.000000,86868.000000,86868.000000,86868.000000,86868.000000,86868.000000,86868.000000,86868.000000,86868.000000,86868.000000,86868.000000,86868.000000,86868.000000,86868.000000,86868.000000,86868.000000,86868.000000,86868.000000,86868.000000,86868.000000
mean,5.266279e+05,13.888198,704.128125,6261.758565,2.133145,26.550168,8.013906,5.436098,0.089953,0.762502,0.034466,0.018568,0.067459,0.073836,0.253534,0.083679,0.042501,0.015794,0.015207,0.983112,0.111468,0.064097,0.033867,0.008242,0.001013,0.000368,0.001002,0.000104,0.000115,0.000046,0.004904
std,3.037828e+05,8.029576,413.364015,3648.712191,0.728047,73.413489,4.377899,1.906950,0.286115,0.425552,0.182424,0.134996,0.250816,0.261506,0.435036,0.276907,0.201731,0.124679,0.122376,0.422363,0.314713,0.244928,0.180889,0.090413,0.031812,0.019190,0.031631,0.010178,0.010729,0.006786,0.069857
min,7.000000e+00,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.644215e+05,7.000000,350.000000,3073.000000,2.000000,10.000000,5.000000,4.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.269835e+05,12.000000,709.000000,6276.000000,2.000000,15.000000,7.000000,5.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.890842e+05,21.000000,1054.000000,9416.000000,2.000000,30.000000,9.000000,6.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.052923e+06,30.000000,1427.000000,12567.000000,8.000000,995.000000,92.000000,32.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,8.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


There seems to be no missing values in the training or testing set because every column has the same amount of rows as the total rows.  

We can now create a new dataframe with the training values and training labels combined.

In [9]:
train_df = pd.merge(train, label, how='inner', on='building_id')

In [10]:
train_df.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
0,802906,6,487,12198,2,30,6,5,t,r,n,f,q,t,d,1,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,3
1,28830,8,900,2812,2,10,8,7,o,r,n,x,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,2
2,94947,21,363,8973,2,10,5,5,t,r,n,f,x,t,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,3
3,590882,22,418,10694,2,10,6,5,t,r,n,f,x,s,d,0,1,0,0,0,0,1,1,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,2
4,201944,11,131,1488,3,30,8,9,t,r,n,f,x,s,d,1,0,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,3


https://medium.com/fuzz/machine-learning-classification-models-3040f71e2529

### EDA

Before looking at the dataset, we will check to see if the labels we are trying to predict is imbalanced.

In [ ]:
train_df['damage_grade'].value_counts()

In [ ]:
sns.countplot('damage_grade', data=train_df)

We will examine the correlation between the variables with a single graph.

In [ ]:
train_df.corr().style.background_gradient(cmap='coolwarm').set_precision(2)

The column with the highest correlation to 'damage_grade', our label, is 'has_structure_mud_mortar_stone' with a value of only 0.29. One of the highest correlations (0.77) is between the columns 'height_percentage' and 'count_floors_pre_eq'. Let's plot those on a scatter plot to see the relationship visually.

In [ ]:
sns.scatterplot(y='height_percentage', x='count_floors_pre_eq', hue='damage_grade', data=train_df)

In [ ]:
sns.catplot(x='foundation_type', y='count_floors_pre_eq', data=train_df, hue='damage_grade', kind='swarm')

Let's see if the 'has_secondary_use' column is correlated with the other 'has_secondary_use' columns because it might just be a broader column than the other ones.

In [ ]:
train_df.filter(like='has_secondary', axis=1).corr().style.background_gradient(cmap='coolwarm').set_precision(2)

There does not actually seem to be very high correlation between any of them except for 'has_secondary_use' and 'has_secondary_use_agriculture'.  

Columns with different data types will need to be manipulated differently. Thus we will create different lists for the categories of data types to make them easier to use.

In [ ]:
# continuous or discrete numeric features
num_features = ['count_floors_pre_eq', 'age', 'area_percentage',
       'height_percentage','count_families']

# categorical features
cat_features = ['land_surface_condition', 'foundation_type',
       'roof_type', 'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration','legal_ownership_status']

# binary features
binary_features = ['has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone',
       'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick',
       'has_superstructure_timber', 'has_superstructure_bamboo',
       'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other','has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
       'has_secondary_use_health_post', 'has_secondary_use_gov_office',
       'has_secondary_use_use_police', 'has_secondary_use_other']

In [ ]:
# function to create count plots
def count_plots(nrows, ncols, feat_list):
    fig, axes = plt.subplots(nrows=nrows, ncols= ncols, figsize=(18,24))
    axes_list = [item for sublist in axes for item in sublist]
    
    for col in feat_list:
        ax = axes_list.pop(0)
        sns.countplot(col, hue='damage_grade', data=train_df, ax=ax)
        ax.legend(loc='upper right', title='damage grade')
    # prevents overlapping
    plt.tight_layout()
    # removes unused subplots
    for ax in axes_list:
        ax.remove()

In [ ]:
count_plots(4,2,cat_features)

In each categorical column, the bar graph is showing some values have significantly more counts than the other. A quick value_counts() of each column can reveal if there is any imbalanced classes.

In [ ]:
for col in cat_features:
    print(col)
    print(train_df[col].value_counts())
    print('\n')

All the columns have some imbalance in the values where one value dominates the total amount.

In [ ]:
plt.figure(figsize=(10,7))
sns.boxplot(x='damage_grade', y='age', data=train_df)

Something funky is happening with the age column. It seems to have outliers that are way higher than the rest. Let's check it out with value counts.  

On the graph, it shows that the majority age are less than 300 and the outliers are all near 1000. Therefore, we will filter out the 'age' coulmn to return only values higher than 400.

In [ ]:
train_df[train_df['age']> 400]['age'].value_counts()

Since the outliers are all the same value and it is so much higher than the rest, we will do some research to see if there truly were buildings about 995 years old in Nepal that was damaged.  

Helpful website: https://www.frontiersin.org/articles/10.3389/fbuil.2015.00008/full

https://www.scmp.com/magazines/post-magazine/article/1880561/restoring-nepals-earthquake-hit-monuments-race-against-time

https://www.nationalgeographic.com/news/2015/04/150427-nepal-earthquake-damage-temples-buddhism-hinduism-world-heritage-monuments-unesco/#close

https://www.frontiersin.org/articles/10.3389/fbuil.2017.00062/full

After reading through various articles, many confirmed that although some buildings were first constructed in the 4th and 5th century (around 300-500AD), many were destroyed in the 1934 earthquake and reconstructed at a later date. The oldest building was said to be about 875 years old. The buildings with 995 as their age could be a typo.  

Since we have a large dataset of about 260000 rows and there are about 1400 of the outlier value, we will simply drop rows where the age is greater than 800.

In [ ]:
train_df = train_df[train_df['age'] < 800]
train_df.head()

A boxplot will be created again to check on the changes.

In [ ]:
sns.boxplot(x='damage_grade', y='age', data=train_df, palette='inferno_r')

Much better! Now let's have a look at a bar graph of 'age' and 'damage_grade' to visualize the average for the three categories.

In [ ]:
sns.barplot(x='damage_grade', y='age', data=train_df, palette='ocean_r')

From the bar graph, we can see that buildings labeled with grade 1 damage is on average younger and the damage grade increases with age. This makes sense since older buildings are more likely to not be up to standard with the building codes.  
Let's take a look at the other numeric features in a histogram plot.

In [ ]:
# function to create box plots
def count_plots(nrows, ncols, feat_list):
    fig, axes = plt.subplots(nrows=nrows, ncols= ncols, figsize=(20,15))
    axes_list = [item for sublist in axes for item in sublist]
    
    for col in feat_list:
        ax = axes_list.pop(0)
        sns.countplot('damage_grade', y=col, data=train_df, ax=ax)
    # prevents overlapping
    plt.tight_layout()
    # removes unused subplots
    for ax in axes_list:
        ax.remove()

In [ ]:
count_plots(2,3,num_features)

### Data Preprocessing

In [ ]:
train_final = pd.get_dummies(train_df[cat_features])
test_final = pd.get_dummies(test[cat_features])

print(train_final.shape)
print(test_final.shape)

In [ ]:
features = np.concatenate([train_final, np.array(train_df[binary_features]), np.array(train_df[num_features])], axis=1)
print('training features shape: ', features.shape)

test_features = np.concatenate([test_final, np.array(test[binary_features]), np.array(test[num_features])], axis=1)
print('test features shape: ', test_features.shape)

In [ ]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler().fit(features[:, -5:])
# features[:, -5:] = scaler.transform(features[:, -5:])

# test_features[:, -5:] = scaler.transform(test_features[:, -5:])

# print(features[:, -5:])
# print(test_features[0:5])

In [ ]:
from sklearn.model_selection import train_test_split

X = features
y = np.array(train_df['damage_grade'])

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=111)
print(x_train.shape)
print(x_test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()

LR.fit(x_train, y_train)

result = LR.predict(x_test)

result